## Load latest headlines

In [ ]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

## Calculate word frequencies

In [ ]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words]
counts = Counter(filtered)
score_df = pd.DataFrame(counts.items(), columns=['word','score']).sort_values('score', ascending=False)
score_df.to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-%M-00')
score_df.to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()

## Rank headlines by score

In [ ]:
word_scores = dict(score_df.values)
latest['score'] = latest['title'].apply(lambda t: sum(word_scores.get(w.lower(),0) for w in re.findall(r'[A-Za-z]+', t)))
ranked = latest.sort_values('score', ascending=False)
ranked.to_csv('rank.csv', index=False)
ranked.to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()

## Next steps